In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs
#from selenium import webdriver
import re
import time
import random

from tqdm.notebook import tqdm
import json

In [2]:
# Specify url
url = 'https://www.cricbuzz.com/live-cricket-scores/43191/ind-vs-zim-42nd-match-super-12-group-2-icc-mens-t20-world-cup-2022/'


In [3]:
response = requests.get(url)

soup = bs(response.content, 'html.parser')
score = soup.findAll("div",attrs={"class","cb-col cb-col-67 cb-scrs-wrp"})

In [4]:
score

[<div class="cb-col cb-col-67 cb-scrs-wrp"> <div class="cb-text-gray cb-font-16"> IND 186/5 (20) </div> <div class="cb-min-bat-rw"> <span class="cb-font-20 text-bold"> ZIM 36/4 (7.2) </span> <span class="cb-font-12 cb-text-gray"> <span class="text-bold">  CRR: </span><span>4.91</span> </span> <span class="cb-font-12 cb-text-gray"> <span class="text-bold">  REQ: </span><span>11.92</span> </span> </div> <div class="cb-text-inprogress">Zimbabwe need 151 runs in 76 balls</div> </div>]

In [5]:
#convert from bs4 to list

def get_lst(bs4_ip):
    bs4_ip_cleaned = [] 
    for x in bs4_ip:
        bs4_ip_cleaned.append(str(x))
    temp = bs4_ip_cleaned[0]
    return str(bs4_ip_cleaned)

In [6]:
converted_lst = get_lst(score)

In [7]:
converted_lst

'[\'<div class="cb-col cb-col-67 cb-scrs-wrp"> <div class="cb-text-gray cb-font-16"> IND 186/5 (20) </div> <div class="cb-min-bat-rw"> <span class="cb-font-20 text-bold"> ZIM 36/4 (7.2) </span> <span class="cb-font-12 cb-text-gray"> <span class="text-bold">\\xa0\\xa0CRR:\\xa0</span><span>4.91</span> </span> <span class="cb-font-12 cb-text-gray"> <span class="text-bold">\\xa0\\xa0REQ:\\xa0</span><span>11.92</span> </span> </div> <div class="cb-text-inprogress">Zimbabwe need 151 runs in 76 balls</div> </div>\']'

In [8]:
bowling_team_cleaned = converted_lst.partition('<div class="cb-col cb-col-67 cb-scrs-wrp"> <div class="cb-text-gray cb-font-16">')[2]
bowling_team_cleaned = bowling_team_cleaned.partition('</div>')[0]     

bowling_team_name = bowling_team_cleaned[1:4]
bowling_team_score = bowling_team_cleaned.partition('/')[0]     
bowling_team_score = re.sub("[^0-9]", "", bowling_team_score)
bowling_team_score = int(bowling_team_score)

In [9]:
bowling_team_name

'IND'

In [10]:
bowling_team_score

186

In [11]:
batting_team_cleaned = converted_lst.partition('<span class="cb-font-20 text-bold">')[2]
batting_team_cleaned = batting_team_cleaned.partition('</span>')[0]

batting_team_name = batting_team_cleaned[1:4]
batting_team_score = batting_team_cleaned.partition('/')[0]
batting_team_score = re.sub("[^0-9]", "", batting_team_score)
batting_team_score = int(batting_team_score)

batting_team_wickets = batting_team_cleaned.partition('/')[2]
batting_team_wickets = batting_team_wickets.partition('(')[0]
batting_team_wickets = batting_team_wickets.replace(' ','')
batting_team_wickets = int(batting_team_wickets)

In [12]:
overs = batting_team_cleaned.partition('(')[2]
overs = overs.partition(')')[0]


overs_played = overs.partition('.')[0]

if '.' in overs:
    balls = overs.partition('.')[2]
else:
    balls = 0


In [13]:
overs_played = int(overs_played)
over_to_balls = 6*overs_played

In [14]:
balls  = int(balls)
balls

2

In [15]:
total_balls = over_to_balls + balls
total_balls

44

In [16]:
balls_left = 120- total_balls
balls_left

76

In [17]:
batting_team_wickets

4

In [18]:
batting_team_wickets = int(batting_team_wickets)
wickets_left = 10-batting_team_wickets

In [19]:
batting_team_name

'ZIM'

In [20]:
batting_team_score

36

In [21]:
runs_left = (bowling_team_score - batting_team_score) + 1
runs_left

151

In [22]:
response = requests.get(url)

soup = bs(response.content, 'html.parser')
stadium = soup.findAll("div",attrs={"class","cb-nav-subhdr cb-font-12"})

stadium_lst = get_lst(stadium)


In [23]:
stadium_lst_cleaned = stadium_lst.partition('itemtype="http://schema.org/Place"')[2]
stadium_lst_cleaned = stadium_lst_cleaned.partition(',')[0]
stadium_lst_cleaned = stadium_lst_cleaned.partition('title="')[2]
stadium_lst_cleaned 

'Melbourne Cricket Ground'

In [24]:
df = pd.DataFrame()

df['batting_team'] = [batting_team_name]
df['bowling_team'] = bowling_team_name
df['city'] = stadium_lst_cleaned
df['runs_left'] = runs_left
df['balls_left'] = balls_left
df['wickets'] = wickets_left
df['total_run_x'] =  bowling_team_score
df['current_score'] =  batting_team_score


#current RR
df['crr'] = (df['current_score']*6)/(120-df['balls_left'])

#required RR
df['rrr'] = (df['runs_left']*6)/df['balls_left']
df['rrr'] = df['rrr'].fillna(0)
df['rrr'] = np.where(df['rrr'].astype(str) == 'inf',111,df['rrr'])

In [25]:
df.head()

,batting_team,bowling_team,city,runs_left,balls_left,wickets,total_run_x,current_score,crr,rrr
0,ZIM,IND,Melbourne Cricket Ground,151,76,6,186,36,4.909091,11.921053


In [26]:
df.to_csv(r'D:\Data Science\Projects\Cricket t20\Data\Prepared\prediction set\test.csv',index=False)